In [1]:
import numpy as np 
np.random.seed(42)
percorso_file = 'problem_r2_500.npy'

# Carica l'array NumPy dal file
try:
    array_caricato = np.load(percorso_file)

    # Stampa l'array e le sue proprietà
    print(f"Array caricato: \n{array_caricato}")
    print(f"Forma (Shape): {array_caricato.shape}")
    print(f"Tipo di dato (Dtype): {array_caricato.dtype}")

except Exception as e:
    print(f"Errore durante il caricamento del file: {e}")

Errore durante il caricamento del file: [Errno 2] No such file or directory: 'problem_r2_500.npy'


In [ ]:
#diagonal matrix

def diagonal(matrix):
    matrix = np.asarray(matrix)
    if matrix.ndim != 2 or matrix.shape[0] != matrix.shape[1]:
        raise ValueError("La matrice deve essere quadrata")
    
    # Deve essere numerica
    if not np.issubdtype(matrix.dtype, np.number):
        raise TypeError("La matrice deve contenere valori numerici")

    # Nessun NaN/Inf
    if not np.all(np.isfinite(matrix)):
        raise ValueError("La matrice contiene NaN o Inf")

    # True se tutti gli elementi fuori diagonale sono zero
    return 1 if np.all(matrix == np.diag(np.diag(matrix))) else 0

diag = diagonal(array_caricato)
print(f"La matrice è diagonale? {diag}")



In [ ]:

# Supponiamo che 'matrice_distanze' sia la tua matrice di adiacenza (N x N)
# dove matrice_distanze[i, j] è la distanza tra la città i e la città j.

def calcola_lunghezza_percorso(percorso: np.ndarray, matrice_distanze: np.ndarray) -> float:
    """Calcola la lunghezza totale di un percorso TSP, includendo il ritorno alla partenza."""
    lunghezza = 0.0
    num_citta = len(percorso)

    # 1. Somma le distanze tra le città consecutive
    for i in range(num_citta):
        citta_attuale = percorso[i]
        
        # La prossima città è l'elemento successivo nel percorso,
        # o la prima città per chiudere il ciclo (TSP)
        citta_successiva = percorso[(i + 1) % num_citta] 
        
        lunghezza += matrice_distanze[citta_attuale, citta_successiva]

    return lunghezza

def funzione_fitness(percorso: np.ndarray, matrice_distanze: np.ndarray) -> float:
    """
    Funzione fitness: 1 / Lunghezza totale del percorso.
    Più corto è il percorso, maggiore è il fitness.
    """
    lunghezza_totale = calcola_lunghezza_percorso(percorso, matrice_distanze)
    
    # Preveniamo la divisione per zero (se la lunghezza fosse 0, anche se non dovrebbe accadere nel TSP)
    if lunghezza_totale == 0:
        return np.inf  # Fitness massimo per un percorso nullo (impossibile)
    
    return 1.0 / lunghezza_totale

In [ ]:
def inizializza_popolazione(dimensioni_popolazione: int, num_citta: int) -> list:
    """Crea una popolazione iniziale di percorsi casuali."""
    popolazione = []
    citta_ids = np.arange(num_citta)
    
    for _ in range(dimensioni_popolazione):
        # np.random.permutation crea una permutazione casuale (un percorso valido)
        nuovo_percorso = np.random.permutation(citta_ids)
        popolazione.append(nuovo_percorso)
        
    return popolazione

In [ ]:
def mutazione_swap(percorso: np.ndarray, tasso_mutazione: float) -> np.ndarray:
    """Scambia due città a caso nel percorso con una data probabilità."""
    percorso_mutato = np.copy(percorso)
    
    if np.random.rand() < tasso_mutazione:
        # Sceglie due indici casuali
        idx1, idx2 = np.random.choice(len(percorso), size=2, replace=False)
        
        # Scambia le città
        percorso_mutato[idx1], percorso_mutato[idx2] = percorso_mutato[idx2], percorso_mutato[idx1]
        
    return percorso_mutato

In [ ]:
# Passaggio 0: Prepara i dati di esempio (Matrice di Distanze)
NUM_CITTA = 500
# Matrice simmetrica casuale (es. distanze euclidee)
matrice_distanze_es = np.random.randint(1, 100, size=(NUM_CITTA, NUM_CITTA))
# Assicura matrice_distanze_es[i, i] = 0
np.fill_diagonal(matrice_distanze_es, 0) 
# Rendi simmetrica (problema simmetrico)
matrice_distanze_es = (matrice_distanze_es + matrice_distanze_es.T) / 2 
matrice_distanze_es = matrice_distanze_es.astype(int)

# --- Parametri dell'AG ---
DIM_POP = 100
NUM_GENERAZIONI = 500
TASSO_MUTAZIONE = 0.05

# Inizializza la popolazione
popolazione = inizializza_popolazione(DIM_POP, NUM_CITTA)
miglior_percorso = None
miglior_fitness = -np.inf
for generazione in range(NUM_GENERAZIONI):
    # 1. Calcola il Fitness per tutta la popolazione
    fitness_values = np.array([funzione_fitness(p, matrice_distanze_es) for p in popolazione])
    
    # Aggiorna il Migliore in Assoluto
    idx_migliore = np.argmax(fitness_values)
    if fitness_values[idx_migliore] > miglior_fitness:
        miglior_fitness = fitness_values[idx_migliore]
        miglior_percorso = popolazione[idx_migliore]

    # 2. Selezione (qui useremo una semplice Elitè e Selezione a Ruota)
    # Per una reale implementazione, qui useresti il crossover e la mutazione
    nuova_popolazione = []
    
    # Manteniamo i due migliori (Elitismo)
    nuova_popolazione.append(popolazione[idx_migliore])
    
    # 3. Creazione della nuova generazione (Selezione, Crossover, Mutazione)
    while len(nuova_popolazione) < DIM_POP:
        # Selezione dei genitori (sostituisci con Selezione a Torneo/Ruota)
        genitore1 = popolazione[np.random.choice(DIM_POP, p=fitness_values/np.sum(fitness_values))]
        genitore2 = popolazione[np.random.choice(DIM_POP, p=fitness_values/np.sum(fitness_values))]
        
        # Applicazione di Crossover (PMX o OX - codice omesso per brevità)
        # figlio = crossover(genitore1, genitore2)
        figlio = genitore1.copy() # Placeholder
        
        # Mutazione
        figlio_mutato = mutazione_swap(figlio, TASSO_MUTAZIONE)
        nuova_popolazione.append(figlio_mutato)

    popolazione = nuova_popolazione

# --- Risultati Finali ---
lunghezza_minima = calcola_lunghezza_percorso(miglior_percorso, matrice_distanze_es)

print(f"✅ Ottimo Percorso Trovato (Generazione {generazione}):")
print(f"   Percorso: {miglior_percorso} (e ritorno alla prima città)")
print(f"   Lunghezza Minima: {lunghezza_minima}")